# Import dependencies

In [ ]:
# To mute annoying warnings in notebook
import warnings

# For datascience
import numpy as np
import pandas as pd

# For graph
import matplotlib.pyplot as plt

# To create iterators
import itertools

from prophet import Prophet
from prophet.plot import (
    plot_plotly,
    add_changepoints_to_plot,
    plot_cross_validation_metric,
)
from prophet.diagnostics import cross_validation, performance_metrics
from datetime import timedelta

warnings.filterwarnings("ignore")

# Getting data, observations
## Get dataset

In [ ]:
# Get dataset from file
df = pd.read_csv(
    "../data/T10Y2Y.csv",
)

# Show dataset head
df.head()

In [ ]:
# Get dataset info
df.info()

In [ ]:
# Get dataset statistics
df.describe()

# Prophet
## Linear model
Dataset has invalid items. Drop them.

In [ ]:
# Replace '.' with NaN
df.replace(
    to_replace=".",
    value=np.NaN,
    inplace=True,
)

# Drop Nans
df.dropna(
    axis=0,
    inplace=True,
)

# Show result
df.describe()

Rename columns and set appropriate data types.

In [ ]:
# Change column names
df.columns = ["ds", "y"]

# Set data type for y to float
df.y = df.y.astype(float)

# Set data type for tima as Timestamp
df.ds = pd.to_datetime(df.ds)

Fit the model.

In [ ]:
# Instantiating a new Prophet object
model_linear = Prophet(
    growth="linear",
)

# Fit
model_linear.fit(df);

Get a suitable dataframe that extends into the future a specified number of days.

In [ ]:
# Get trend using the extrapolated generative model
future = model_linear.make_future_dataframe(periods=365)

# Show tail of trend
future.tail()

Make prediction

In [ ]:
# Assign each row in future a predicted value - yhat
forecast = model_linear.predict(future)

# Get dataframe with predicted values, components and uncertainty intervals.
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].tail()

Plot the forecast

In [ ]:
fig1 = model_linear.plot(
    fcst=forecast,
    xlabel="Time",
    ylabel="Yield predicted value",
    include_legend=True,
)

plt.title("Yield forecast");

We see the century oscillation as well as year. Let's look closer.

In [ ]:
fig_interactive = plot_plotly(model_linear, forecast)

# Update scatter trace to change color to white
fig_interactive.update_traces(marker=dict(color="white"))

Plot trend, holidays, weekly seasonality, and yearly seasonality.

In [ ]:
# Show the forecast components.
fig2 = model_linear.plot_components(
    fcst=forecast,
    plot_cap=True,
)

Yield as a rule rise on weekends, significantly fall in the end of the summer. Moreover, we see global minimum in the begning of 2023.

## Remove outliers

Add change points

In [ ]:
# Locate and remove outliers
df.loc[
    (df["ds"] > "2023-10-05") & (df["ds"] < "2023-11-16"),
    "y",
] = None

# Get new fit
cleared_model = Prophet().fit(df)

# Plot cleared data
fig_cleared = cleared_model.plot(cleared_model.predict(future))

Forcast a bit straightened.

## Find change points

Add change points

In [ ]:
# Make new figure
change_points_figure = model_linear.plot(forecast)

# Add change points based on threshold value
a = add_changepoints_to_plot(
    ax=change_points_figure.gca(),
    m=model_linear,
    fcst=forecast,
    threshold=3,
)

For the threshold value of 3 we get 3 clear trends.

## Hyperparameter training

Add change points

In [ ]:
df_cv = None

# Get parameter grid
param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.1, 0.5],
    "seasonality_prior_scale": [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [
    dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())
]

# Store the RMSEs for each params here
rmses = []

# Use cross validation to evaluate all parameters
for params in all_params:
    # Fit model with given params
    model_grid = Prophet(**params).fit(df)

    df_cv = cross_validation(
        model=model_grid,
        initial="730 days",
        period="365 days",
        horizon="365 days",
        parallel="processes",
    )

    df_p = performance_metrics(
        df=df_cv,
        rolling_window=1,
    )

    rmses.append(df_p["rmse"].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)

tuning_results["rmse"] = rmses

tuning_results

In [ ]:
best_params = all_params[np.argmin(rmses)]

best_params

In [ ]:
df_cv

In [ ]:
if df_cv is not None:
    try:
        fig_cv_metrics = plot_cross_validation_metric(
            df_cv=df_cv,
            metric="rmse",
            rolling_window=0.1,
        )

    except Exception as e:
        print(f"Wrong TypeError: --- {e} --- Issue with function suspected.")

else:
    print("df_cv is None")

## Logistic model - Saturation

Get sub-period to discover.

In [ ]:
# Get current date
current_datetime = pd.Timestamp.now()

# Filter time period
filtered_df = df[
    (df["ds"] > current_datetime - timedelta(days=3.95 * 365))
    & (df["ds"] < current_datetime - timedelta(days=3.85 * 365))
]

Set logistic parameters.

In [ ]:
# Set saturation
saturation = 0.55

# Add constant saturation values as a column
filtered_df["cap"] = saturation

Get logistic model.

In [ ]:
# Get model with weekly oscillations dropped to have more smooth curve
model_log = Prophet(
    growth="logistic",
    weekly_seasonality=False,
)

# Fit model
model_log.fit(filtered_df);

Make future dataframe.

In [ ]:
# Get dataframe
future_log = model_log.make_future_dataframe(periods=30)

# Add saturation column to predicted dataframe
future_log["cap"] = saturation

Make prediction and plot.

In [ ]:
# Predict
forecast_log = model_log.predict(future_log)

# Plot forecast
fig_log = model_log.plot(
    fcst=forecast_log,
    include_legend=True,
)

plt.title("Prophet Logistic Growth Forecast");

Yield value .8 is forcast in 1 month. Let's compare the result with real values

In [ ]:
# Filter time period in original dataset
filtered_df = df[
    (df["ds"] > current_datetime - timedelta(days=3.95 * 365))
    & (df["ds"] < current_datetime - timedelta(days=3.8 * 365))
]

In [ ]:
# Plot data for period of forcast
plot = filtered_df.plot(
    x="ds",
    y="y",
    figsize=(10, 6),
    grid=True,
)

constant_value = 0.55

plt.axhline(
    y=constant_value,
    color="r",
    linestyle="--",
);

Logistic forcast is wrong. Real trand differs from forcast one.

# Summary
1. Prophet package used to discover example dataset on trends.
2. Hyperparameters tuned.
2. LogisiticRegression used to make the prediction